# Image Segmentation

In [1]:
import ipywidgets as widgets
import numpy as np
from PIL import Image
import io
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2

import sys
sys.path.insert(0,'.')

from src.utils import keras_losses as loss

In [2]:
def setup_model(path='notebooks/models/unet2_weights.h5'):
    model = loss.make_model(image_size=(224, 224))
    model.compile(loss=loss.BCE_dice, optimizer='sgd', metrics=['accuracy'])
    model.load_weights(path)
    return model

In [3]:
model = setup_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
def img_inference(img, model):
    img = np.array(img)[None, :, :, :]
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    pre = model.predict(img)

    pre = np.argmax(pre, axis=-1).squeeze()
    
    return pre

In [5]:
#learn_inf = load_learner('export(1).pkl')

def on_click_classify(change):
    img = Image.open(io.BytesIO(btn_upload.data[-1]))
    img = img.resize((224, 224))
    pred = img_inference(img, model)
    
    pred = pred.squeeze()
    with image_pl: 
        display(img)

    with prediction_pl: 
        display(Image.fromarray(pred.astype(int)))


btn_upload = widgets.FileUpload()
image_pl = widgets.Output()
prediction_pl = widgets.Output()
lbl_pred = widgets.Label()
lbl_pred.value = "Select an image for Classification!"
btn_run = widgets.Button(description='Classify')
btn_run.on_click(on_click_classify)

In [6]:
widgets.VBox([btn_upload, btn_run])

In [7]:
widgets.HBox([image_pl, prediction_pl])